In [305]:
import pandas as pd
import numpy as np
import datetime
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import pickle
import winsound

In [446]:
#data = pd.read_csv("korean_air_2020_01.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_02.csv")
#data = pd.read_csv("korean_air_2020_03.csv")
#data = pd.read_csv("korean_air_2020_04.csv")
#data = pd.read_csv("korean_air_2020_05.csv")
#data = pd.read_csv("korean_air_2020_6.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_07.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_08.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_09.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_10.csv")
#data = pd.read_csv("korean_air_data_11~12.csv")

#data = pd.read_csv("naver_2020_1.csv")
#data = pd.read_csv("naver_2020_2.csv")
#data = pd.read_csv("naver_2020_3.csv")
#data = pd.read_csv("naver_2020_4.csv")
#data = pd.read_csv("naver_2020_5.csv")
#data = pd.read_csv("naver_2020_6.csv")
#data = pd.read_csv("naver_2020_7.csv")
#data = pd.read_csv("naver_2020_8.csv")
#data = pd.read_csv("naver_2020_9.csv")
#data = pd.read_csv("naver_2020_10.csv")

data = pd.read_csv("lg_chem_2020_01.csv")
#data = pd.read_csv("lg_chem_2020_02.csv")
#data = pd.read_csv("lg_chem_2020_03.csv")

#data = pd.read_csv("lg_chem_2020_04.csv")
#data = pd.read_csv("lg_chem_2020_05.csv")
#data = pd.read_csv("lg_chem_2020_06.csv")
#data = pd.read_csv("lg_chem_2020_07.csv")
#data = pd.read_csv("lg_chem_2020_08.csv")

#data = pd.read_csv("lg_chem_2020_09.csv")
#data = pd.read_csv("lg_chem_2020_10.csv")
#data = pd.read_csv("lg_chem_2020_11~12.csv")



In [447]:
data = data.dropna()
data.columns = ["a","title","text","repoter","spon", "date"]
date = data.date

In [448]:
#q불용어 지정
#기자이름
repoter = data["repoter"].unique

In [449]:
#신문사 이름
spon = data["spon"].unique

In [450]:
#대표적인 조사
stopwords = ['의','가','이','권','은','들','는','좀','잘', \
             '걍','과','도','을','것','를','등','으로','자',\
             '에','등','와','한','하다''것','로']

In [451]:
#불용어 병합
stopwords.extend(data.repoter.values)
stopwords.extend(data.spon.values)

In [452]:
len(data)

390

In [453]:
# 한글 외의 모든 데이터 제거
data = data.text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


okt = Okt()
X_train = []
X_train2 = []

for sen in data :
    temp_X = []
    
    # 토큰화
    temp_X = okt.morphs(sen, stem = True) 
    
    # 불용어 제거
    temp_X = [word for word in temp_X if not word in stopwords] 
    
    #모든 토큰들을 리스트에 기사별로 저장
    X_train.append(temp_X)
    
    #모든 토큰들을  하나의 리스트에 저장
    X_train2.extend(temp_X)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3

# 단어의 수
total_cnt = len(tokenizer.word_index)

# 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
rare_cnt = 0 

# 훈련 데이터의 전체 단어 빈도수 총 합
total_freq = 0 

# 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총합
rare_freq = 0 

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt/total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq/total_freq)*100)

# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
vocab_size = total_cnt - rare_cnt
print('단어 집합의 크기 :',vocab_size)

단어 집합(vocabulary)의 크기 : 8235
등장 빈도가 2번 이하인 희귀 단어의 수: 3956
단어 집합에서 희귀 단어의 비율: 48.03885853066181
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 3.6091794158553547
단어 집합의 크기 : 4279


In [454]:
nouns = []
str = ''

for k in range(0,len(X_train)):
    for i in range(0,len(X_train[k])):
        str = str + X_train[k][i] + ' '
    nouns.append(str)

In [455]:
tfidf_df = pd.DataFrame()

tfIdfVectorizer = TfidfVectorizer(use_idf=True)
tfidf = tfIdfVectorizer.fit_transform(nouns)

for i in range(0,len(X_train)):
    df = pd.DataFrame(tfidf[i].T.toarray(), \
                      index=tfIdfVectorizer.get_feature_names(), \
                      columns=["text{}".format(i)])
    tfidf_df = pd.concat([tfidf_df,df],axis = 1)

X_train_tfidf = []
X_train_tfidf2 = []

for i in range(0,len(X_train)):
    idx = tfidf_df['text{}'.format(i)] > 0.01
    result = tfidf_df[idx].index.tolist()
    X_train_tfidf.append(result)
    X_train_tfidf2.extend(result)

In [456]:
#요일 구하기
date2 = []
weekday = []

for x in date:
    x = x.replace("-","/")
    D = x.split("/")
    y = int(D[0])
    y_string = D[0]
    if len(D[1]) < 2 :
        D[1] = "0" + D[1]
        m = int(D[1])
        m_string = D[1]
        
    else : 
        m = int(D[1])
        m_string = D[1]
    
    if len(D[2]) < 2 :
        D[2] = "0" + D[2]
        d = int(D[1])
        d_string = D[2]
        
    else : 
        d = int(D[2])
        d_string = D[2]
        
    date2.append(y_string + "/" + m_string + "/" + d_string)
    weekday.append(['MON','TUE','WED','THU','FRI','SAT','SUN']\
                   [datetime.date(y,m,d).weekday()])

In [457]:
date = date2

In [458]:
DATE = []

for x in date:
    if x not in DATE:
        DATE.append(x)

In [459]:
#증감 데이터 불러오기
increase_r = pd.read_csv("LG화학 대비처리.csv")
#increase_r = pd.read_csv("대한항공 대비처리.csv")
#increase_r = pd.read_csv("naver 대비처리.csv")

In [460]:
increase = increase_r[["년/월/일","증감"]]

In [461]:
weekday_frame = pd.DataFrame( {"date": date, "weekday": weekday})

In [462]:
for x in range(0,len(weekday_frame)):
    if weekday_frame.weekday[x] == "SAT":
        monday = DATE.index(weekday_frame.date[x]) - 2
        weekday_frame.date[x] = DATE[monday]
        weekday_frame.weekday[x] == "MON"
            
    else :
        if weekday_frame.weekday[x] == "SUN":
            monday = DATE.index(weekday_frame.date[x]) - 1
            weekday_frame.date[x] = DATE[monday]
            weekday_frame.weekday[x] == "MON"
                
        else : pass
                

In [463]:
weekday_frame = weekday_frame.drop_duplicates()

In [464]:
#weekday_frame.to_csv("weekday_naver.csv")
#weekday_frame.to_csv("weekday_daehan.csv")

In [465]:
#날자를 중심으로 데이터 병합
increase = increase.rename(columns = {"년/월/일" : "date"})

In [466]:
dic_frame = pd.DataFrame({"data" : X_train_tfidf, "date" : date})

In [467]:
increase2 = pd.merge(weekday_frame, increase , on = "date")

In [468]:
del increase2["weekday"]

In [469]:
increase2 = increase2.drop_duplicates()

In [470]:
dic_frame2 = pd.merge(dic_frame, increase2, on = "date", how = "left")

In [471]:
dic = dic_frame2.fillna(0)

In [472]:
#딕셔너리 key 생성
#key_list = []

#for v in X_train_tfidf2:
#    if v not in key_list:
#        key_list.append(v)

In [473]:
#예상 소요 시간 300분
#사전 만들기
#dictionary = {"<UNK>": {"횟수": 0, "점수": 0, "극성정도": 0}}

#for k in key_list:  #키 리스트안에 있는 단어가 언급이 되는 경우
#    count = 0
#    score = 0
#    for x in range(0,len(dic)):  #한 기사씩 검사한다.
#        d = dic.loc[x,:].data
        
        
        #count 는 전체 언급된 횟수,score는 증감의 총합,pority는 총합의 평균
#        if k in d:
#            count = count + d.count(k)
#            score = score + (dic.loc[x,:]["증감"] * d.count(k))
               
#        else:
            
#            pass
        
#    try:
        
#        polarity  = score/count
        
#    except ZeroDivisionError as e:
        
#        print(x,k)
        
#    dictionary[k] = {"횟수" : count, "점수" : score, "극성정도" : polarity}  

In [474]:
#딕셔너리 저장
#with open('lg_chem_10','wb') as fw:
#    pickle.dump(dictionary, fw)

In [475]:
#최종 dic계산
with open('daehan_fin.pickle', 'rb') as f:
    daehan = pickle.load(f)
with open('lg_chem_fin.pickle', 'rb') as f:
    lg_chem = pickle.load(f)    

In [535]:
#기사별 점수 계산
scores = []
miss = []
for x in range (0, len(dic)):
    key = set(dic.data[x])
    score = 0
    count = 0
    
    for i in key:
        count += Counter(dic.data[x])[i]
        
    try:    
            
        score += (daehan[i]["극성정도"] * Counter(dic.data[x])[i])
        score = score/count
        scores.append(score)
            
    except KeyError :
        
        score += 0
        score = score/count
        scores.append(score)
        
        
        
    except ZeroDivisionError :
        
        print(x,k)
        miss.append(x)
            

In [536]:
len(scores)

390

In [537]:
#이상치 제거
dic.drop(miss, inplace = True)
dic["score"] = scores

In [538]:
#날자별 점수 평균계산
score_data = dic.groupby(['date'])["score"].mean()

#날자별 평균 데이터 저장
score_data.to_csv("lg_chem_1_score.csv")

In [539]:
#알람
winsound.PlaySound("SystemExit", winsound.SND_ALIAS)

In [542]:
dic

,data,date,증감,score
0,"[계약, 공급, 공식, 공장, 글로벌, 기록, 기자, 까지, 나오다, 난징, 넘어서...",2020/01/31,-1.0,0.000000
1,"[계약, 공급, 공식, 공장, 규모, 글로벌, 기록, 기자, 까지, 나오다, 난징,...",2020/01/31,-1.0,0.000000
2,"[가량, 가치, 경제, 계약, 공급, 공식, 공장, 과는, 국내, 규모, 그간, 글...",2020/01/31,-1.0,0.000000
3,"[가량, 가치, 각각, 강세, 같다, 개인, 거래, 경제, 계약, 공급, 공식, 공...",2020/01/31,-1.0,-0.001861
4,"[가능성, 가량, 가운데, 가치, 각각, 감안, 감염, 감염증, 강세, 같다, 개국...",2020/01/31,-1.0,-0.000937
...,...,...,...,...
385,"[가능성, 가다, 가동, 가장, 각각, 감소, 강세, 강조, 강화, 같다, 개발, ...",2020/01/02,0.0,-0.007659
386,"[가격, 가능성, 가다, 가동, 가장, 각각, 감소, 강세, 강조, 강화, 같다, ...",2020/01/02,0.0,-0.007631
387,"[가격, 가능성, 가다, 가동, 가장, 각각, 감소, 강세, 강조, 강화, 같다, ...",2020/01/02,0.0,-0.007631
388,"[가격, 가능성, 가다, 가동, 가장, 각각, 감소, 강세, 강조, 강화, 같다, ...",2020/01/02,0.0,-0.007659


In [544]:
dic.score.unique()

array([ 0.        , -0.00186128, -0.00093656, -0.00082417, -0.00140614,
       -0.00124714, -0.00109727, -0.00098115, -0.00096332, -0.00113766,
       -0.00179104, -0.00164801, -0.00157161, -0.00181155, -0.00166536,
       -0.00161438, -0.00120414, -0.00157683, -0.00167713, -0.00159271,
       -0.00173221, -0.00190613, -0.00185401, -0.00183964, -0.00186861,
       -0.00183254, -0.00182549, -0.00189094, -0.00189851, -0.001876  ,
       -0.0018468 , -0.00180466, -0.00177763, -0.00176441, -0.00179783,
       -0.00175788, -0.00174495, -0.001771  , -0.00181849, -0.00188344,
       -0.00804376, -0.00810636, -0.00807494, -0.00801282, -0.00798212,
       -0.00795165, -0.00789141, -0.00786164, -0.00792142, -0.00783208,
       -0.00780275, -0.00777363, -0.00774473, -0.00768758, -0.00771605,
       -0.00765931, -0.00763126, -0.00757576, -0.00760341, -0.00754831,
       -0.00752106])